In [38]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
import os
import yaml
import itertools
import shutil
from datetime import datetime
import sys
# # load svensson links
# sv_links = pd.read_excel('/Users/quinnmackay/Documents/GitHub/BICC/Data Storage/Tiepoints/Svensson Links NGRIP-NEEM-GIP-GISP-EDML-EDC-WD-DJ-TALDICE.xls',
#                          sheet_name=1, nrows=305, usecols=[0,2,3,4,5,6,7,], names=['NGRIP', 'GRIP', 'GISP2', 'EDML', 'EDC', 'WDC', 'DF'])

In [ ]:
sv_links = pd.read_excel('/Users/quinnmackay/Documents/GitHub/BICC/Data Storage/Tiepoints/DF_SPC_tiepoints_forQuinn.xlsx', skiprows=34, usecols=[0,2,5], names=['DF1', 'DF2', 'WDC']) 

comment_add = 'From Ikumi SP/DF synchronization, email sent on Nov 8. Also includes interpolated DF2, DF1 tiepoints.'

sv_links2 = sv_links[pd.isna(sv_links['DF1'])]
sv_links3 = sv_links[~pd.isna(sv_links['DF1'])]
sv_links2['DF1'] = np.interp(sv_links2['DF2'], sv_links3['DF2'], sv_links3['DF1'])

sv_links = sv_links2[sv_links2['DF1'] <= 979]

sv_links = sv_links[['DF1', 'WDC']]
sv_links = sv_links.reset_index(drop=True)
sv_links.rename(columns={'DF1':'DF'}, inplace=True)
sv_links

/var/folders/cd/1mh45k3j19g5g_kxkz_17xvc0000gn/T/ipykernel_18466/2365839959.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sv_links2['DF1'] = np.interp(sv_links2['DF2'], sv_links3['DF2'], sv_links3['DF1'])


,DF,WDC
0,8.548118,46.503905
1,18.805498,92.790000
2,21.443415,105.300000
3,34.073282,168.979141
4,41.413666,208.925000
...,...,...
116,855.325252,3182.673517
117,897.561550,3240.941950
118,944.006654,3300.893085
119,961.445907,3320.870000


In [40]:
backups=True
folder='BICC2025'
error=20

if backups == True:
    # add backup of BICC

    source_folder = f'/Users/quinnmackay/Documents/GitHub/BICC/Paleochrono BICC Work/Paleochrono BICC Experiment/{folder}'
    backup_root = '/Users/quinnmackay/Documents/GitHub/BICC/Paleochrono BICC Work/Paleochrono BICC Experiment/Backups'

    # Create timestamp
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    backup_folder = os.path.join(backup_root, f'BICC_Backup_{timestamp}')

    # Make sure the backup root exists
    os.makedirs(backup_root, exist_ok=True)

    # Copy the folder
    shutil.copytree(source_folder, backup_folder)

    print(f"Backup completed: {backup_folder}")
else:
    print("Backups not enabled.")


Backup completed: /Users/quinnmackay/Documents/GitHub/BICC/Paleochrono BICC Work/Paleochrono BICC Experiment/Backups/BICC_Backup_20251111_133858


In [41]:
#load cores list from params
params = f'/Users/quinnmackay/Documents/GitHub/BICC/Paleochrono BICC Work/Paleochrono BICC Experiment/{folder}/parameters.yml'
with open(params, 'r') as f:
    first_line = f.readline()
params_load = yaml.safe_load(first_line)
list_sites = params_load['list_sites']

# get all link combos
pairs = [f"{a}-{b}" for a, b in itertools.combinations(list_sites, 2)]

#get all combos possible from the svensson links
subset = list(sv_links.columns)
valid_pairs = [p for p in pairs if all(site in subset for site in p.split('-'))]

In [ ]:
#now, create all the link files
sv_synchros = {}
sv_comments = {}
for pair in valid_pairs:
    site_a, site_b = pair.split('-')

    # Drop NaN rows to get only valid shared tiepoints
    sv_synchros[pair] = sv_links[[site_a, site_b]].dropna()
    sv_synchros[pair].columns = ['depth1', 'depth2']
    sv_synchros[pair]['age_unc'] = error
    sv_synchros[pair]['comment'] = np.nan  # Initialize age column with NaN

    sv_comments[pair] = [comment_add]

#now load existing synchros
os.chdir(f'/Users/quinnmackay/Documents/GitHub/BICC/Paleochrono BICC Work/Paleochrono BICC Experiment/{folder}')
existing_synchros = {}
existing_comments = {}

for pair in valid_pairs:
    folder_path = pair
    file_path = os.path.join(folder_path, "iceice_synchro_horizons.txt")

    os.makedirs(folder_path, exist_ok=True)

    if os.path.exists(file_path):

        comments = []
        with open(file_path, 'r') as f: # store existing comments to be re-added
            for line in f:
                if line.startswith('#'):
                    comments.append(line.rstrip('\n'))  # keeps tabs intact  # store comment lines without newline
        existing_comments[pair] = comments

        try:
            synchros = pd.read_csv(file_path, sep='\t', comment='#')
        except Exception as e:
            print(f"Error loading {pair}: {e}")
            print(f'File Path: {file_path}')
            sys.exit()

        existing_synchros[pair] = synchros
        print(f"Loaded {pair}: {len(synchros)} rows")
    else:
        print(f"Skipped {pair} — file not found")

Loaded DF-WDC: 533 rows


In [ ]:
tolerance = 0.10  # meters

for pair in valid_pairs:

    if pair in existing_synchros:
        
        overlaps = 0
        new_ties = 0
        error_overlaps = 0

        existing = existing_synchros[pair].copy(deep=True)
        new = sv_synchros[pair].copy(deep=True)

        total_existing = len(existing)

        # Start with originals (priority)
        combined = existing.copy(deep=True)

        # Append only existing tiepoints that are NOT close to any new tiepoint
        for idx, row in new.iterrows():

            mask_either = ( #check for either column within tolerance. Does NOT append if either is close.
                (abs(existing.iloc[:, 0] - row.iloc[0]) <= tolerance) |
                (abs(existing.iloc[:, 1] - row.iloc[1]) <= tolerance)
            )
            mask_both = ( #check for both columns to spit out error if needed
                (abs(existing.iloc[:, 0] - row.iloc[0]) <= tolerance) &
                (abs(existing.iloc[:, 1] - row.iloc[1]) <= tolerance)
            )

            if not mask_either.any():
                combined = pd.concat([combined, row.to_frame().T], ignore_index=True)
                new_ties+=1
            else:
                overlaps += 1

            if mask_either.any() and not mask_both.any():
                print(f"Warning: Partial overlap detected for pair {pair} at new tiepoint {row.values}. One column is within {tolerance} of an existing tiepoint, while other is not. The tiepoint was not added to avoid conflicts.")
                error_overlaps += 1

        #duplicate check
        before = len(combined)
        combined = combined.drop_duplicates(ignore_index=True)
        after = len(combined)

        if after < before:
            print(f"Warning: {before - after} duplicate tiepoint(s) found and removed in pair {pair}.")

        folder_path = pair
        file_path = os.path.join(folder_path, "iceice_synchro_horizons.txt")
        os.makedirs(folder_path, exist_ok=True)

        # --- Gather comments ---
        new_comments = sv_comments.get(pair, [])

        with open(file_path, 'w') as f:
            # Write new comments first
            if new_ties > 0:
                for line in new_comments:
                    f.write(f"{line}\n")
            # Add original comments
            for line in existing_comments.get(pair, []):
                f.write(f"{line}\n")
            # Write merged DataFrame below
            combined.to_csv(f, sep='\t', index=False)
        print(f"Saved merged synchro for {pair} with rows to {file_path}\nThere were {total_existing} existing rows, {overlaps} overlaps ({error_overlaps} error overlaps), and {new_ties} new ties.")

    else:
        folder_path = pair
        file_path = os.path.join(folder_path, "iceice_synchro_horizons.txt")
        os.makedirs(folder_path, exist_ok=True)

        combined = sv_synchros[pair].copy(deep=True) # svensson links load
        new_comments = sv_comments.get(pair, [])

        with open(file_path, 'w') as f:
            # Write new comments first
            for line in new_comments:
                f.write(f"#{line}\n")
            # Write merged DataFrame below
            combined.to_csv(f, sep='\t', index=False)

        print(f"No existing synchros for {pair}, saving new synchro to {file_path}")


Saved merged synchro for DF-WDC with rows to DF-WDC/iceice_synchro_horizons.txt
There were 533 existing rows, 121 overlaps (13 error overlaps), and 0 new ties.
